In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
class AssismentData():
    def __init__(self):
        self.data = pd.read_csv("/content/drive/My Drive/DKT/2015_100_skill_builders_main_problems.csv")

        self.data.dropna()

        self.data["user_id"], _ = pd.factorize(self.data["user_id"])
        self.data["sequence_id"], _ = pd.factorize(self.data["sequence_id"])
        self.data["skills"] = self.data.apply(lambda x: x.sequence_id * 2 if x.correct == 0.0 else x.sequence_id * 2 + 1, axis=1)

        self.data = self.data.drop(columns="log_id", axis=1)

        self.data = self.data.groupby("user_id").filter(lambda q: len(q) > 1).copy()

        self.seq = self.data.groupby('user_id').apply(
            lambda r: (
                r["sequence_id"].values,
                r['skills'].values,
                r['correct'].values
            )
        )

        self.train = self.seq.sample(frac=0.8)
        self.test = self.seq[~self.seq.index.isin(self.train.index)]


    def datasetReturn(self, data, shuffle=None, batch_size=50, val_data=None):
        dataset = tf.data.Dataset.from_generator(lambda: data, output_types=(tf.int32, tf.int32, tf.int32))

        if shuffle:
            dataset = dataset.shuffle(buffer_size=shuffle)

        MASK_VALUE = -1
        dataset = dataset.padded_batch(
            batch_size=batch_size,
            padding_values=(MASK_VALUE, MASK_VALUE, MASK_VALUE),
            padded_shapes=([None], [None], [None]),
            drop_remainder=True
        )

        return dataset

In [9]:
ass = AssismentData()
train_data,test_data = ass.datasetReturn(ass.train),ass.datasetReturn(ass.test)
val_log = 'log/val'
train_loss_log = 'log/train'
summary_writer = tf.summary.create_file_writer(val_log)


In [10]:

total_skills_correctness = 200
total_skills = 100
embedding_size = 100
batchsize = 50
M = 50
class DKVMNcell(tf.keras.layers.AbstractRNNCell):
    def __init__(self, units, **kwargs):
        self.units = units
        super(DKVMNcell, self).__init__(**kwargs)
        self.Mv = self.add_weight(shape=(M, embedding_size),
                                  initializer='random_normal',
                                  trainable=True)
        self.Mv = tf.expand_dims(self.Mv,axis=0)
    @property
    def state_size(self):
        return self.units

    def call(self, w_attention, erase_signal_mul, add_signal_mul, states):
        """
        :param w_attention: 这个应该是concept矩阵计算后的注意力权重
        :param erase_signal: erase标志
        :param add_signal: add标志
        :param states: Mk矩阵
        :return: r，Mv
        """
        # 读
        # w_attention.shape (50,50) state.shape (50,50,100)
        r = tf.matmul(tf.expand_dims(w_attention,axis=1),states)
        
        # print(r.shape)(50,1,100)
        r = r[:,0,:]
        
        # 写
        states = states * erase_signal_mul + add_signal_mul
       
        return r, states


class DKVMN(tf.keras.models.Model):
    def __init__(self):
        super(DKVMN, self).__init__()
        # 掩码层
        self.mask = tf.keras.layers.Masking(mask_value=-1)
        # 题目嵌入
        self.exercise_embedding = tf.keras.layers.Embedding(total_skills, embedding_size)
        # 题目对错嵌入
        self.exercise_correctness_embedding = tf.keras.layers.Embedding(total_skills_correctness, embedding_size)

        self.cell = DKVMNcell(10)

        self.Mk = self.add_weight(shape=(M, embedding_size),
                                  initializer='random_normal',
                                  trainable=True)

        self.erase = tf.keras.layers.Dense(embedding_size)
        self.add = tf.keras.layers.Dense(embedding_size, activation="tanh")
        self.r = tf.keras.layers.Dense(embedding_size, activation="tanh")
        self.p = tf.keras.layers.Dense(2,activation="sigmoid")

    def call(self, skillid, skill_correctness,correctness):
        shape = skillid.shape
        skill_correctness = tf.expand_dims(skill_correctness, axis=-1)
        skillid = tf.expand_dims(skillid, axis=-1)
        # 掩码
        skillid = self.mask(skillid)
        skill_correctness = self.mask(skill_correctness)
        # 映射
        skill_embedding = self.exercise_embedding(skillid)
        skill_correctness_embedding = self.exercise_correctness_embedding(skill_correctness) #（batch,seqlen,embeddingsize）
        skill_correctness_embedding = tf.squeeze(skill_correctness_embedding, axis=2) 
        skill_embedding = tf.squeeze(skill_embedding, axis=2)
        # tensorlist_batch = tf.TensorArray(dtype=tf.float32,size=0,dynamic_size=True)
        # for k in range(skill_correctness_embedding.shape[0]):
        #   tensorlist = tf.TensorArray(tf.float32,size=0,dynamic_size=True)
        #   for i in range(skill_correctness_embedding.shape[1]):
        #     # 做对拼接1 ，做错拼接0
        #     w=tensorlist.write(i,tf.cond(correctness[k,i],lambda:tf.zeros_like(skill_correctness_embedding[k,i]),lambda:tf.ones_like(skill_correctness_embedding[k,i])))
        #     w.mark_used()
        #   w=tensorlist_batch.write(k,tensorlist.stack())
        #   w.mark_used()
        # # 拼接后skill_correctness_embedding
        # skill_correctness_embedding = tf.concat([skill_correctness_embedding,tensorlist_batch.stack()],axis=-1)
        
        # 产生 注意力权重
        w_attention = tf.matmul(skill_embedding, tf.expand_dims(tf.transpose(self.Mk), axis=0))
        w_attention = tf.nn.softmax(w_attention)

        #  遗忘 和 更新 Mv的过程
        erase_signal = self.erase(skill_correctness_embedding)
        add_signal = self.add(skill_correctness_embedding)

        erase_signal_mul = 1 - tf.expand_dims(w_attention, axis=-1) * tf.expand_dims(erase_signal, axis=2)
        add_signal_mul = tf.expand_dims(w_attention, axis=-1) * tf.expand_dims(add_signal, axis=2)
        # 遗忘和更新Mv
        # batch个 Mv
        states = self.cell.Mv
        for i in range(batchsize)[1:]:
          states = tf.concat([states,self.cell.Mv],axis=0)
    
        cell_out_list = tf.TensorArray(size=0,dynamic_size=True,dtype=tf.float32)

        for i in range(shape[1]):
            r,states = self.cell(w_attention[:,i],erase_signal_mul[:,i],add_signal_mul[:,i],states)
            w = cell_out_list.write(i,tf.expand_dims(r,axis=1))
            w.mark_used()
        f = cell_out_list.read(0)
        for i in range(shape[1])[1:]:
            f = tf.concat([f,cell_out_list.read(i)],axis=1)
        
        r = tf.concat([f,skill_embedding], axis=-1)
        loss = tf.nn.softmax(self.p(self.r(r)))
        return loss



In [12]:

def test_one_step(skillid,skill_correctness,correctness):
    probility = dkvmn(skillid, skill_correctness,correctness)

    mask = 1 - tf.cast(tf.equal(correctness, -1), tf.int32)

    mask = tf.squeeze(mask)
    # mask掉
    probility = tf.boolean_mask(probility, mask)
    label = tf.boolean_mask(correctness, mask)

    label = tf.one_hot(label, depth=2)
   
    vauc.update_state(label,probility)

def train_one_step(skillid,skill_correctness,correctness):
    with tf.GradientTape() as tape:
        probility = dkvmn(skillid,skill_correctness,correctness)
        mask = 1 - tf.cast(tf.equal(correctness,-1),tf.int32)
        mask = tf.squeeze(mask)
        # mask 掉
        probility = tf.boolean_mask(probility,mask)
        label = tf.boolean_mask(correctness,mask)
        
        label = tf.one_hot(label, depth=2)
        # 求bc
        bc.update_state(label,probility)

        loss = tf.losses.categorical_crossentropy(label,probility)
       
        auc.update_state(label,probility)
        
        gradients = tape.gradient(loss, dkvmn.trainable_variables)
        # 反向传播，自动微分计算
        optimizer.apply_gradients(zip(gradients, dkvmn.trainable_variables))


In [13]:
dkvmn = DKVMN()
bc = tf.metrics.CategoricalCrossentropy()
auc = tf.metrics.AUC()
vauc = tf.metrics.AUC()
optimizer = tf.optimizers.Adam(learning_rate=0.01)

In [ ]:
import time
for epoch in range(10):
    start = time.time()
    train_data = train_data.shuffle(32)
    auc.reset_states()
    vauc.reset_states()
    bc.reset_states()
    for  s, v, l in train_data.as_numpy_iterator():
        train_one_step(s, v, l)

    for s, v, l in test_data.as_numpy_iterator():
        test_one_step(s, v, l)
    print(time.time()-start)
    with summary_writer.as_default():
        tf.summary.scalar('train_auc', auc.result(), step=epoch)
        tf.summary.scalar('val_auc', vauc.result(), step=epoch)

    print(bc.result(), auc.result(), vauc.result())

In [ ]:
vauc.reset_states()
for s, v, l in test_data.as_numpy_iterator():
    test_one_step(s, v, l)
print(vauc.result())

tf.Tensor(0.7753244, shape=(), dtype=float32)
